In [9]:
# importing necessary libraries

import torch
import torchvision
import torch.utils.data as dataloader
import torch.nn as nn

In [4]:
from torchvision import datasets, transforms

# import dataset
# Define transforms
transform = transforms.Compose([transforms.ToTensor()])

# Download and load MNIST dataset
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
mnist_val = datasets.MNIST(root='./data', train=False, download=True, transform=transform)  
transform = torchvision.transforms.Compose([transforms.ToTensor()])

In [6]:
# define batches 
train_loader = dataloader.DataLoader(mnist_train, batch_size=64, shuffle=True)
val_loader = dataloader.DataLoader(mnist_val, batch_size=64, shuffle=True)

In [8]:
# define variable 
num_classes = 10
batch_size =64
num_channels = 1
image_size = 28
patch_size = 7
num_patches = (image_size // patch_size) ** 2
embedding_dim = 64
attention_heads =2
transformer_blocks = 4
learning_rate = 0.001
epochs = 5
mlp_hidden_nodes = 128



In [ ]:
### part 1 : patch embedding
### part 2 : transformer encoder
### part 3: mlp head
### transformer class

### part 1 : patch embedding

In [ ]:
class PatchEmbedding(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.patch_embed=nn.Conv2d(num_channels,embedding_dim,kernel_size=patch_size,stride=patch_size, )
        